# PROJECT : ANALYZING CIA FACTBOOK DATA USING SQL

# Introduction
In this project, we will be working with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information for countries like: population - The population as of 2015. population_growth - The annual population growth rate, as a percentage. area - The total land and water area.

The database, factbook.db used in this analysis can be downloaded [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db). We will be working with SQLite in this project to explore and analyse the database.



## Connecting to the database and viewing all the tables in the db

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


From the above cell, we see that there is only one table in the database named facts that is relevant to our analysis task in hand. Let's view the first 5 rows in the the table - facts


In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* name - The name of the country.
* area - The total land and sea area of the country.
* population - The country's population.
* population_growth- The country's population growth as a percentage.
* birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
* death_rate - The country's death rate, or the number of death a year per 1,000 people.
* area- The country's total area (both land and water).
* area_land - The country's land area in square kilometers.
* area_water - The country's water area in square kilometers.

Let's start by calculating some summary statistics 

# Summary Statistics

In [4]:
%%sql
SELECT MIN(population), 
       MAX(population), 
       MIN(population_growth), 
       MAX(population_growth)
FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the above cell:

* There's a country with a population of 0
* There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values


# Exploring Outliers

In [5]:
%%sql 
SELECT *
FROM facts
WHERE population == (SELECT MIN(population)
       FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html)

In [6]:
%%sql 
SELECT *
FROM facts
WHERE population == (SELECT MAX(population)
       FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It also seems like the table above contains a row for the whole world, which explains the population of over 7.2 billion.


Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.


# Summary Statistics revisited

In [17]:
%%sql
SELECT MIN(population), 
       MAX(population), 
       MIN(population_growth), 
       MAX(population_growth)
FROM facts
WHERE name != 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion

# Exploring Average Population and Area
Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet ie the World row.

In [8]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts
WHERE name <> 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers


# Finding Densely Populated Countries
To finish, we'll build on the query above to find countries that are densely populated. We'll identify countries that have:

* Above average values for population.
* Below average values for area.

In [9]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts)
AND area < (SELECT AVG(area)
                   FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


Some of these countries are generally known to be densely populated, so we have confidence in our results!

But, let's answer the question below to see how it compares with the above result table


## What countries have the highest population/area ratio?

In [18]:
%%sql
SELECT name, CAST(population AS float)/CAST(area AS float) AS pa_ratio
  FROM facts
WHERE name != 'World'
ORDER BY pa_ratio DESC
LIMIT 10;

Done.


name,pa_ratio
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259


# Exploring other questions


## Which country has the most people?

In [11]:
%%sql
SELECT name, MAX(population)
FROM facts
WHERE name != 'World'

Done.


name,MAX(population)
China,1367485388


## Which country has the highest growth rate?

In [12]:
%%sql
SELECT name, MAX(population_growth)
FROM facts
WHERE name != 'World'

Done.


name,MAX(population_growth)
South Sudan,4.02


## Which countries have the highest ratios of Water to land?

In [19]:
%%sql
SELECT name, CAST(area_water AS float)/CAST(area_land AS float) AS wl_ratio
  FROM facts
WHERE name != 'World'
ORDER BY wl_ratio DESC
LIMIT 10;

Done.


name,wl_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


## Which countries have more Water than Land?

In [20]:
%%sql
SELECT name, area_water, area_land
  FROM facts
WHERE area_water > area_land;

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


## Which countries will add the most people to their population next year? ie highest population growth (by number) by next year

In [15]:
%%sql
SELECT
    name,
    population,
    population_growth,
    CAST((population_growth * population)/100 AS FLOAT)
    AS predicted_population_growth
FROM facts
WHERE name <> 'World' 
ORDER BY predicted_population_growth DESC
LIMIT 10;

Done.


name,population,population_growth,predicted_population_growth
India,1251695584,1.22,15270686.1248
China,1367485388,0.45,6153684.246
Nigeria,181562056,2.45,4448270.372
Pakistan,199085847,1.46,2906653.3662
Ethiopia,99465819,2.89,2874562.1691
Bangladesh,168957745,1.6,2703323.92
United States,321368864,0.78,2506677.1392
Indonesia,255993674,0.92,2355141.8008000003
"Congo, Democratic Republic of the",79375136,2.45,1944690.8320000002
Philippines,100998376,1.61,1626073.8536


## Which countries have a higher death rate than birth rate?

In [16]:
%%sql
SELECT name, birth_rate, death_rate
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC;

Done.


name,birth_rate,death_rate
Ukraine,10.72,14.46
Bulgaria,8.92,14.44
Latvia,10.0,14.31
Lithuania,10.1,14.27
Russia,11.6,13.69
Serbia,9.08,13.66
Belarus,10.7,13.36
Hungary,9.16,12.73
Moldova,12.0,12.59
Estonia,10.51,12.4


# Conclusion

The main objective of this project though was to understand how the basic SQL code works and what information could be extracted using SQL. We analyzed the CIA Factbook Data and answered some summary questions about the database and discovered some interesting facts about the world's countries:
  
 *  Macau, located in China, is a territory with the highest population density in the world.
 *  British Indian Ocean Territory is a territory in the Indian Ocean with the highest water-to-land ration (more than 900 times)
 * Ukraine has the highest death rate in the world
 * China has the highest population of almost 1.4 billion people
 * South Sudan has the highest population growth rate in the world.
